In [2]:
import pandas as pd

factify_train = pd.read_csv("/Users/sakdag/Downloads/public_folder/train.csv")

In [5]:
factify_train["Category"].value_counts()

Category
Support_Text               7000
Insufficient_Multimodal    7000
Support_Multimodal         7000
Insufficient_Text          7000
Refute                     7000
Name: count, dtype: int64

In [6]:
factify_val = pd.read_csv("/Users/sakdag/Downloads/public_folder/val.csv")

In [7]:
factify_val["Category"].value_counts()

Category
Support_Multimodal         1500
Refute                     1500
Support_Text               1500
Insufficient_Text          1500
Insufficient_Multimodal    1500
Name: count, dtype: int64

In [11]:
import pandas as pd
from tqdm import tqdm
from langdetect import detect, DetectorFactory
from langdetect.lang_detect_exception import LangDetectException

# Ensuring consistent language detection
DetectorFactory.seed = 0

# Example DataFrame
# factify_train = pd.DataFrame({'claim': ['This is an English sentence.', 'Ceci est une phrase française.', 'Este es un texto en español.', ...]})

def detect_language(text):
    try:
        return detect(text)
    except LangDetectException:
        return "unknown"

# Use tqdm to show progress
tqdm.pandas(desc="Detecting languages")

# Apply language detection with progress bar
factify_train['language'] = factify_train['document'].progress_apply(detect_language)

# Get counts per language
language_counts = factify_train['language'].value_counts()

# Display the results
print(language_counts)


Detecting languages: 100%|██████████| 35000/35000 [03:00<00:00, 194.15it/s]

language
en         34969
nl            11
hi             7
de             6
es             2
it             1
fr             1
unknown        1
id             1
bn             1
Name: count, dtype: int64


In [2]:
import pickle
import os

def modify_keys_in_pkl(input_file: str, output_file: str):
    """
    Reads a .pkl file, modifies its keys by replacing backslashes (\\) with slashes (/),
    and writes the updated data back to a new .pkl file.

    Args:
        input_file (str): Path to the input .pkl file.
        output_file (str): Path to the output .pkl file.
    """
    # Ensure the input file exists
    if not os.path.exists(input_file):
        raise FileNotFoundError(f"Input file {input_file} does not exist.")

    # Read the data from the input .pkl file
    with open(input_file, 'rb') as file:
        data = pickle.load(file)

    # Ensure the data is a dictionary
    if not isinstance(data, dict):
        raise ValueError("The .pkl file must contain a dictionary.")

    # Modify the keys in the dictionary
    modified_data = {
        (key.replace('\\', '/') if isinstance(key, str) else key): value
        for key, value in data.items()
    }

    # Write the modified data back to a new .pkl file
    with open(output_file, 'wb') as file:
        pickle.dump(modified_data, file)

    print(f"Modified keys saved to {output_file}")

In [5]:
from src.utils.path_utils import get_project_root

project_root = get_project_root()
old_pkl = os.path.join(project_root, "evidence_feature_old.pkl")
new_pk = os.path.join(project_root, "evidence_feature.pkl")

In [6]:
modify_keys_in_pkl(old_pkl, new_pk)

Modified keys saved to /Users/sakdag/Desktop/masters/multimodal-misinformation-detection/evidence_feature.pkl


In [13]:
with open(new_pk, 'rb') as file:
    data = pickle.load(file)
    print(list(data.keys())[0])
    print(data[list(data.keys())[0]])
    print(list(data.keys())[-1])
    print(data[list(data.keys())[-1]])
    print(len(data))

data/raw/factify/extracted/images/test/0_evidence.jpg
tensor([0.0000, 0.0000, 0.0750,  ..., 0.2198, 0.0000, 0.0027])
data/raw/factify/extracted/images/train/9_evidence.jpg
tensor([0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0383, 0.0000])
41256


In [32]:
from src.evidence.text2text_retrieval import SemanticSimilarity
from PIL import Image
from src.demo.app import Evidence
from src.utils.path_utils import get_project_root
import os
import pandas as pd


PROJECT_ROOT = get_project_root()


query = "Michael Bloomberg said that Donald Trump was known among millionaires as a 'con artist,' and among business owners as a 'cheat.'. a close up of a person in a suit and tie with a quote"
data_dir = os.path.join(PROJECT_ROOT, "data", "preprocessed")
train_csv_path = os.path.join(data_dir, "train_enriched.csv")
test_csv_path = os.path.join(data_dir, "test_enriched.csv")
train_embeddings_file = os.path.join(PROJECT_ROOT, "train_embeddings.h5")
test_embeddings_file = os.path.join(PROJECT_ROOT, "test_embeddings.h5")
similarity =SemanticSimilarity(
    train_embeddings_file=train_embeddings_file,
    test_embeddings_file=test_embeddings_file,
    train_csv_path=train_csv_path,
    test_csv_path=test_csv_path,
)
evidences = []
# Perform semantic search across both train and test datasets
results = similarity.search(query=query, top_k=5)

# Retrieve evidence rows based on the search results
for evidence_id, score in results:
    # Determine whether the ID belongs to train or test set
    if evidence_id.startswith("train_"):
        df = similarity.train_csv
    elif evidence_id.startswith("test_"):
        df = similarity.test_csv
    else:
        continue  # Skip invalid IDs

    # Extract the row by ID
    row = df[df["id"] == int(evidence_id.split("_")[1])].iloc[0]
    evidence_text = row.get("evidence_enriched")
    evidence_image_caption = row.get("evidence_image_caption")
    evidence_image_path = row.get("evidence_image")
    evidence_image = None

    # Load the image if a valid path is provided
    if pd.notna(evidence_image_path):
        full_image_path = os.path.join(PROJECT_ROOT, evidence_image_path)
        try:
            evidence_image = Image.open(full_image_path).convert("RGB")
        except Exception as e:
            print(f"Failed to load image {evidence_image_path}: {e}")

        # Create an Evidence object
        evidences.append(
            Evidence(
                text=evidence_text,
                image=evidence_image,
                caption=evidence_image_caption,
                evidence_id=evidence_id,
            )
        )



len(hits_train) = 25
len(hits_test): 25
[('train_12005', -1.2907099), ('train_3005', -1.2907099), ('train_6005', -1.2907099), ('train_20892', -1.5562872), ('train_14787', -2.8103993), ('train_26765', -3.5438893), ('train_28534', -3.5438893), ('train_34754', -3.5438893), ('train_22006', -3.5993924), ('train_23133', -3.6540892), ('train_25594', -3.8778205), ('train_14790', -4.199101), ('train_27141', -4.468935), ('train_31941', -4.588285), ('train_13872', -4.710619), ('train_15856', -4.728036), ('train_7936', -4.728036), ('train_3976', -4.728036), ('train_31590', -5.2371297), ('train_6360', -5.6940365), ('train_3180', -5.6940365), ('train_12720', -5.6940365), ('train_9806', -5.9297), ('train_4906', -5.9297), ('train_31416', -7.693228), ('test_179', 6.086358), ('test_4663', 0.3863513), ('test_7100', -1.2288148), ('test_7414', -1.6169428), ('test_4824', -3.881856), ('test_6779', -4.113426), ('test_2505', -4.217946), ('test_1129', -4.440675), ('test_2280', -4.5340633), ('test_1675', -4.5864

In [35]:
evidences

[Evidence(evidence_id='test_179', evidence='A graphic featuring an image of former New York mayor Michael Bloomberg next to a quote ostensibly uttered by him about Republican presidential nominee Donald Trump, referring to the GOP candidate as “con artist” and a “cheat,” has been widely circulated on social media: The text of this image first started circulating\xa0in July 2016, shortly after Bloomberg made an appearance at the Democratic National Convention to endorse Hillary Clinton. Bloomberg issued several remarks critical of Donald Trump during his DNC speech, including referring to his candidacy as a “con” and mocking his business acumen, but he did not say the words attributed to him above. A full transcript of Bloomberg’s speech can be found here, while the relevant portion has been reproduced below: Nonetheless, some newspapers misquoted Bloomberg and reported that he had referred to Trump as a “con artist”: It was Florida senator Marco Rubio who referred to Trump (more than o

In [24]:
row = similarity.test_csv[similarity.test_csv["id"] == 179].iloc[0]

In [25]:
row

id                                                                       179
claim                      Michael Bloomberg said that Donald Trump was k...
claim_image_original       https://www.snopes.com/tachyon/2016/09/bloombe...
evidence                   A graphic featuring an image of former New Yor...
evidence_image_original    https://www.snopes.com/tachyon/2016/09/bloombe...
category                                                              Refute
claim_ocr                  Michael Bloomberg on Donald Trump\nI have know...
evidence_ocr                                                               月
claim_image                data/raw/factify/extracted/images/test/179_cla...
evidence_image             data/raw/factify/extracted/images/test/179_evi...
claim_image_caption        a close up of a person in a suit and tie with ...
evidence_image_caption     arafed man in a suit and tie standing in an of...
claim_enriched             Michael Bloomberg said that Donald Trump was k...

In [26]:
evidence_image_path = row["evidence_image"]

In [27]:
evidence_image_path

'data/raw/factify/extracted/images/test/179_evidence.jpg'

In [31]:
results[0][0].split("_")[1]

'179'

In [49]:
similarity.train_csv.iloc[24612]

id                                                                     24612
claim                      Multiple people injured in massive stampede as...
claim_image_original       http://pbs.twimg.com/amplify_video_thumb/11129...
evidence                   \n  The killing of Nipsey Hussle on Sunday has...
evidence_image_original    http://pbs.twimg.com/media/D3I6moNXQAAO1lX.jpg...
category                                             Insufficient_Multimodal
claim_ocr                                     C\nINSURANCE\nabc\nNEWS\nTRIBE
evidence_ocr                  Drapka\nESSENT\n***\n100\nARDS\n10-\nORY\nMOEN
claim_image                data/raw/factify/extracted/images/train/24612_...
evidence_image             data/raw/factify/extracted/images/train/24612_...
claim_image_caption        there are many people standing outside of a st...
evidence_image_caption     there is a man standing next to a group of chi...
claim_enriched             Multiple people injured in massive stampede as...

In [47]:
from src.evidence.im2im_retrieval import ImageCorpus
from PIL import Image
from src.demo.app import Evidence
from src.utils.path_utils import get_project_root
import os
import pandas as pd


query_image_path = os.path.join(
    PROJECT_ROOT,
    "data",
    "raw",
    "factify",
    "extracted",
    "images",
    "train",
    "1_claim.jpg",
)
image_features = os.path.join(PROJECT_ROOT, "evidence_features.pkl")
image_corpus = ImageCorpus(image_features)
evidences = []
# Perform semantic search across both train and test datasets
results = image_corpus.retrieve_similar_images(query_image_path, top_k=5)

# Retrieve evidence rows based on the search results
for evidence_path, score in results:
    evidence_id = evidence_path.split("/")[-1].split("_")[0]
    # Determine whether the ID belongs to train or test set
    if "train" in evidence_path:
        data_dir = os.path.join(PROJECT_ROOT, "data", "preprocessed")
        train_csv_path = os.path.join(data_dir, "train_enriched.csv")
        df = pd.read_csv(train_csv_path)
    elif "test" in evidence_path:
        data_dir = os.path.join(PROJECT_ROOT, "data", "preprocessed")
        train_csv_path = os.path.join(data_dir, "test_enriched.csv")
        df = pd.read_csv(train_csv_path)
    else:
        continue  # Skip invalid IDs

    # Extract the row by ID
    row = df[df["id"] == int(evidence_id)].iloc[0]
    evidence_text = row.get("evidence_enriched")
    evidence_image_caption = row.get("evidence_image_caption")
    evidence_image_path = row.get("evidence_image")
    evidence_image = None

    print(evidence_text)
    print(evidence_image_caption)
    print(evidence_image_path)

    if pd.notna(evidence_image_path):
        full_image_path = os.path.join(PROJECT_ROOT, evidence_image_path)
        try:
            evidence_image = Image.open(full_image_path).convert("RGB")
        except Exception as e:
            print(f"Failed to load image {evidence_image_path}: {e}")

    # Create an Evidence object
    evidences.append(
        Evidence(
            text=evidence_text,
            image=evidence_image,
            caption=evidence_image_caption,
            evidence_id=evidence_id,
        )
    )


Sep 08, 2021-Wednesday
 

Metro cities -  Delhi, Mumbai,
											Chennai,
											Kolkata

Other cities -  Noida,
											Gurgaon,
											Bengaluru, Hyderabad, Bhopal
											, Chandigarh , Dehradun,
											Indore,
											Jaipur,
											Lucknow,
											Patna,
											Ranchi

											Powered by 
Six employees of Mumbai's Taj hotel test positive for coronavirusIndiGo employee dies of coronavirus infectionTelangana to extend lockdown till April 30Maharashtra reports 187 new Covid-19 cases, tally reaches to 1,761Chhattisgarh govt makes wearing masks in public places compulsoryNew York City schools closed for rest of year amid Covid-19 pandemicKarnataka extends lockdown for two weeks with some relaxations10 new Covid-19 cases reported in Kerala today: Kerala CMMP govt allows use of MLA funds for battling coronavirusUS reports highest daily virus toll as Easter opens in lockdownWe have been successful in preventing Covid-19 spread to some extent: CM Udd

In [48]:
evidences

[Evidence(evidence_id='32475', evidence='\nSep 08, 2021-Wednesday\n \n\nMetro cities -  Delhi, Mumbai,\n\t\t\t\t\t\t\t\t\t\t\tChennai,\n\t\t\t\t\t\t\t\t\t\t\tKolkata\n\nOther cities -  Noida,\n\t\t\t\t\t\t\t\t\t\t\tGurgaon,\n\t\t\t\t\t\t\t\t\t\t\tBengaluru, Hyderabad, Bhopal\n\t\t\t\t\t\t\t\t\t\t\t, Chandigarh , Dehradun,\n\t\t\t\t\t\t\t\t\t\t\tIndore,\n\t\t\t\t\t\t\t\t\t\t\tJaipur,\n\t\t\t\t\t\t\t\t\t\t\tLucknow,\n\t\t\t\t\t\t\t\t\t\t\tPatna,\n\t\t\t\t\t\t\t\t\t\t\tRanchi\n\n\t\t\t\t\t\t\t\t\t\t\tPowered by \nSix employees of Mumbai\'s Taj hotel test positive for coronavirusIndiGo employee dies of coronavirus infectionTelangana to extend lockdown till April 30Maharashtra reports 187 new Covid-19 cases, tally reaches to 1,761Chhattisgarh govt makes wearing masks in public places compulsoryNew York City schools closed for rest of year amid Covid-19 pandemicKarnataka extends lockdown for two weeks with some relaxations10 new Covid-19 cases reported in Kerala today: Kerala CMMP govt allow

['Over 45,000 new #COVID19 cases, 496 deaths take India’s tally to 8.22 million\n\nhttps://t.co/whWw7M2MG2 https://t.co/tkuLMdsl1N', "#ElectionsWithHT | Voters wait outside a polling station in West Bengal's Barrackpore to cast their vote.\n\nVoting in 51 constituencies across 7 states for #Phase5 to begin shortly.\n\nLIVE updates: \nhttps://t.co/IulJrJVpBE\n\nPics: ANI https://t.co/6REcM7tnWW", 'Karnataka: Defence Minister @nsitharaman arrives at polling booth 54 in Jayanagar of Bangalore South Parliamentary constituency to cast her vote (ANI)\n\nFollow LIVE updates: https://t.co/GB4nYl0g6m\n\n#ElectionsWithHT \n#LokSabhaElections2019 https://t.co/7cMdMPbhcK', 'Trump wants an us-versus-them fight, and sees it as his most viable path to a second term. https://t.co/EmsKEFYrUE', 'Democratic @SenMarkey says that not having John Bolton as a witness would amount to a “cover-up” and if Republicans refuse to call on him “this entire trial will be viewed as a historical joke.” https://t.co/ejN